In [ ]:
import pandas as pd
import numpy as np
import math
import copy

#Naive Bayes Classifiers

##data import and preprocessing

### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

### preprocessing(normalize)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-465-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-465-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

###split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
data_org = data.iloc[:, 1:]
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


##class

In [ ]:
import pandas as pd
import numpy as np
import copy
class Naive_Bayes_Classifier():
  '''binary classifier of Naive Bayes Classifier
  data must have feature and labe, and label should put at the last columnsof data.
  '''
  def __init__(self, data, data_org):
    self.a = data_org
    self.data_org = data_org.iloc[:, :len(self.a.columns)-1]
    self.data = data  #feature+label
    self.x = data.iloc[:, :len(self.data.columns)-1]  #feature
    self.y = data.iloc[:, len(self.data.columns)-1]  #label
    self.x_0 = data[data["is_claim"] == 0].iloc[:, :len(self.data.columns)-1]
    self.x_1 = data[data["is_claim"] == 1].iloc[:, :len(self.data.columns)-1]

  # count all object in all features
  def feature(self):
    self.feature_num = len(self.data_org.columns)  #feature 個數
    self.feature_content = {}   #預設各feature的內容物個數為0(all label)
    for i in range(self.feature_num):
      feature = {}
      for ii in range(len(self.data_org[self.data_org.columns[i]].value_counts())):
        feature[self.data_org[self.data_org.columns[i]].unique()[ii]] = 0
      self.feature_content[self.data_org.columns[i]] = feature

    self.feature_content_0 = {} #各feature的內容物(label == 0)
    self.feature_content_1 = {} #各feature的內容物(label == 1)
    for i in range(self.feature_num):
      self.feature_0 = copy.deepcopy(self.feature_content[self.x.columns[i]]) 
      self.feature_1 = copy.deepcopy(self.feature_content[self.x.columns[i]]) 
      for ii in range(len(self.x_0[self.x_0.columns[i]].value_counts())):
        self.feature_0[self.x_0[self.x_0.columns[i]].unique()[ii]] = self.x_0[self.x_0.columns[i]].value_counts()[self.x_0[self.x_0.columns[i]].unique()[ii]]
      self.feature_content_0[self.x_0.columns[i]] = self.feature_0
      for ii in range(len(self.x_1[self.x_1.columns[i]].value_counts())):
        self.feature_1[self.x_1[self.x_1.columns[i]].unique()[ii]] = self.x_1[self.x_1.columns[i]].value_counts()[self.x_1[self.x_1.columns[i]].unique()[ii]]
      self.feature_content_1[self.x_1.columns[i]] = self.feature_1
    return self.feature_content_0, self.feature_content_1

  def fit(self):
    self.feature() 
  
  def predict(self, test_data):
    prediction = []
    for i in range(len(test_data)):
      p0 = len(self.x_0)/len(self.x)
      for ii in range(self.feature_num):
        p0 = p0*(self.feature_content_0[test_data.columns[ii]][test_data.iloc[i][ii]]/len(self.x_0))
      p1 = len(self.x_1)/len(self.x)
      for ii in range(self.feature_num):
        p1 = p1*(self.feature_content_1[test_data.columns[ii]][test_data.iloc[i][ii]]/len(self.x_1))
      if p0 > p1:
          predict = 0
      else:
          predict = 1
      prediction.append(predict)
    return prediction

##fit

In [ ]:
NB = Naive_Bayes_Classifier(train, data_org)

In [ ]:
NB.fit()

In [ ]:
test_data = test.iloc[:, :len(test.columns)-1]

In [ ]:
predict = NB.predict(test_data)

In [ ]:
def accuracy(y, y1):
  right = 0
  wrong = 0
  for i in range(len(y)):
    if y[i] == y1[i]:
      right += 1
    elif y[i] != y1[i]:
      wrong += 1
  acc = right/(right+wrong)
  return acc
accuracy(test["is_claim"], predict)

0.9296832332058984

#Random Forest

##data import and preprocessing

### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

### preprocessing(normalize)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-476-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-476-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

###split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
data_org = data.iloc[:, 1:]
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


##class

In [ ]:
class Random_Forest():
  def __init__(self, data, n_estimator):
    self.data_org = data
    self.data = data.iloc[:, 30:]
    self.n_estimator = n_estimator

  def fit(self):
    self.forest = {}
    for i in range(self.n_estimator):
      data_temp = self.Booststrap(self.data)
      tree = self.decision_tree(data_temp)
      self.forest[i] = tree
    
        
  def predict(self, test_data):
    self.data_org["predict"] = 999
    predicted = []
    for iii in range(len(test_data)):
      for i in range(self.n_estimator):
        predict_tree = []
        p = 0
        for ii in range(len(self.forest[i])):
          if test_data[test_data.columns[ii]][iii] == 0:
            p = p - self.forest[i][test_data.columns[ii]][0]
          elif test_data[test_data.columns[ii]][iii] == 1:
            p = p + self.forest[i][test_data.columns[ii]][1]
        if p < 0:
          predict_tree.append(0)
        else:
          predict_tree.append(1)
      if (max(predict_tree, key = predict_tree.count)) == 0:
        predict = 0
      elif (max(predict_tree, key = predict_tree.count)) == 1:
        predict = 1
      predicted.append(predict)
    return predicted
  
  #calculate accuracy of prediction
  def accuracy(self):
    right = 0
    wrong = 0
    for i in range(len(self.data_org)):
      if self.data_org.iloc[i][-2] == self.data_org.iloc[i][-1]:
        right += 1
      elif self.data_org.iloc[i][-2] != self.data_org.iloc[i][-1]:
        wrong += 1
    self.acc = right/(right+wrong)
    return self.acc

  def decision_tree(self, data):
    tree = {}
    for i in range(len(self.data.columns)):
      tree[self.data.columns[i]] = {}
      tree[self.data.columns[i]][0] = 0
      tree[self.data.columns[i]][1] = 0

    #node1
    split_index, split0, split1, split_data = self.node_split(data)
    tree[split_index] = {}
    if len(split0) != 0:
      tree[split_index][0] = len(split0[split0["is_claim"] == 0]) / len(split_data)
    else:
      tree[split_index][0] = 0
    if len(split1) != 0:
      tree[split_index][1] = len(split1[split1["is_claim"] == 1]) / len(split_data)
    else:
      tree[split_index][1] = 0

    #node2
    split_index_0, split0_0, split1_0, split_data_0 = self.node_split(split0)
    tree[split_index_0] = {}
    if len(split0_0) != 0:
      tree[split_index_0][0] = len(split0_0[split0_0["is_claim"] == 0]) / len(split_data_0)
    else:
      tree[split_index_0][0] = 0
    if len(split1_0) != 0:
      tree[split_index_0][1] = len(split1_0[split1_0["is_claim"] == 1]) / len(split_data_0)
    else:
      tree[split_index_0][1] = 0

    #node3
    split_index_1, split0_1, split1_1, split_data_1 = self.node_split(split1)
    tree[split_index_1] = {}
    if len(split0_1) != 0:
      tree[split_index_1][0] = len(split0_1[split0_1["is_claim"] == 0]) / len(split_data_1)
    else:
      tree[split_index_1][0] = 0
    if len(split1_1) != 0:
      tree[split_index_1][1] = len(split1_1[split1_1["is_claim"] == 1]) / len(split_data_1)
    else:
      tree[split_index_1][1] = 0
    return tree

  def node_split(self, data):
    E = self.Information_Gain_E(data)
    split_index = [i for i in E if E[i]==min(E.values())][0]
    split_0 = data[data[[i for i in E if E[i]==min(E.values())][0]] == 0].drop(columns = split_index)
    split_1 = data[data[[i for i in E if E[i]==min(E.values())][0]] == 1].drop(columns = split_index)
    split_data = data.drop(columns = split_index)
    return split_index, split_0, split_1, split_data

  def Information_Gain_E(self, data):
    x, y, x_0, x_1 = self.xy(data)
    E ={}
    feature_content_0, feature_content_1 = self.feature(data)
    for i in range(len(x.columns)):
      e = 0
      for ii in range(len(self.feature_content[x.columns[i]])):
        p = feature_content_1[x.columns[i]][x[x.columns[i]].unique()[ii]]
        n = feature_content_0[x.columns[i]][x[x.columns[i]].unique()[ii]]
        e += self.Information_Gain_I(p, n)
      E[x.columns[i]] = e
    return E 

  def Information_Gain_I(self, p, n):
    if p == 0:
      a = 0
    else:
      a = -(p/(p+n))*math.log((p/(p+n)), 2)
    if n == 0:
      b = 0
    else:
      b = -(n/(p+n))*math.log((n/(p+n)), 2)
    I = a+b
    return I

  def feature(self, data):
    x, y, x_0, x_1 = self.xy(data)
    feature_num = len(x.columns)
    self.feature_content = {}   #預設各feature的內容物個數為0(all label)
    for i in range(feature_num):
      feature = {}
      for ii in range(len(x[x.columns[i]].value_counts())):
        feature[x[x.columns[i]].unique()[ii]] = 0
      self.feature_content[x.columns[i]] = feature

    feature_content_0 = {} #各feature的內容物(label == 0)
    feature_content_1 = {} #各feature的內容物(label == 1)
    for i in range(feature_num):
      feature_0 = copy.deepcopy(self.feature_content[x.columns[i]]) 
      feature_1 = copy.deepcopy(self.feature_content[x.columns[i]]) 
      for ii in range(len(x_0[x_0.columns[i]].value_counts())):
        feature_0[x_0[x_0.columns[i]].unique()[ii]] = x_0[x_0.columns[i]].value_counts()[x_0[x_0.columns[i]].unique()[ii]]
      feature_content_0[x_0.columns[i]] = feature_0
      for ii in range(len(x_1[x_1.columns[i]].value_counts())):
        feature_1[x_1[x_1.columns[i]].unique()[ii]] = x_1[x_1.columns[i]].value_counts()[x_1[x_1.columns[i]].unique()[ii]]
      feature_content_1[x_1.columns[i]] = feature_1
    return feature_content_0, feature_content_1 

  def Booststrap(self, data):
    data_temp = self.feature_generator(self.data_generator(data))
    return data_temp

  def data_generator(self, data):
    data_indexes = np.arange(len(data)//3)
    np.random.shuffle(data_indexes)
    generator_data = pd.DataFrame()
    for k in data_indexes:
      generator_data = pd.concat([generator_data, data.loc[k:k]])
    return generator_data

  def feature_generator(self, data):
    feature_indexes = np.arange(len(data.columns)-1)
    np.random.shuffle(feature_indexes)
    feature_indexes = feature_indexes[:3]
    generator_feature = pd.DataFrame()
    for k in feature_indexes:
      generator_feature[data.columns[k]] = data[data.columns[k]]
    generator_feature["is_claim"] = data["is_claim"]
    return generator_feature

  def xy(self, data):
    x = data.iloc[:, :len(data.columns)-1]  #feature
    y = data.iloc[:, len(data.columns)-1]  #label
    x_0 = data[data["is_claim"] == 0].iloc[:, :len(data.columns)-1]
    x_1 = data[data["is_claim"] == 1].iloc[:, :len(data.columns)-1]
    return x, y, x_0, x_1

##fit

In [ ]:
RF = Random_Forest(train, 20)

In [ ]:
RF.fit()

In [ ]:
test_data = test.iloc[:, 30:]

In [ ]:
predict = RF.predict(test_data)

In [ ]:
def accuracy(y, y1):
  right = 0
  wrong = 0
  for i in range(len(y)):
    if y[i] == y1[i]:
      right += 1
    elif y[i] != y1[i]:
      wrong += 1
  acc = right/(right+wrong)
  return acc
accuracy(test["is_claim"], predict)

0.662411250682687

#Random Forest (By scikit learn)

##data import and preprocessing

### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

### preprocessing(normalize)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-487-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-487-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

###split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
train = data.iloc[index[:int(len(data)/2)], 1:]
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:]
val = data.iloc[index[int(len(data)*3/4):], 1:]
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


##random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier()

In [ ]:
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]

In [ ]:
X_train = pd.get_dummies(X_train)

In [ ]:
forest.fit(X_train,y_train.values)

RandomForestClassifier()

In [ ]:
predict = forest.predict(X_train)

In [ ]:
right = 0
wrong = 0
for i in range(len(predict)):
  if y_train.values[i] == predict[i]:
    right += 1
  else:
    wrong += 4
accuracy = right/(right+wrong)
accuracy

0.8871426339634432

#GBM

##XGBoost

https://ithelp.ithome.com.tw/articles/10273094?sc=iThomeR

###data import and preprocessing

#### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

#### preprocessing(normalize)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-518-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-518-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

####split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


###fit

In [ ]:
from xgboost import XGBClassifier
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)

In [ ]:
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)

In [ ]:
X_test = test.iloc[:, 1:len(test.columns)-1]
y_test = test.iloc[:, len(test.columns)-1]
X_test = pd.get_dummies(X_test)

In [ ]:
xgboostModel.fit(X_train, y_train)

XGBClassifier(learning_rate=0.3)

In [ ]:
predicted = xgboostModel.predict(X_train)

In [ ]:
predicted

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
xgboostModel.score(X_train,y_train)

0.9334038776624796

##LightGBM

https://ithelp.ithome.com.tw/articles/10274577

###data import and preprocessing

#### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

#### preprocessing(normalize)

In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-531-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-531-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


####split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


###fit

In [ ]:
import lightgbm as lgb
lgb = lgb.LGBMClassifier(is_unbalance=True)

In [ ]:
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)

In [ ]:
X_test = test.iloc[:, 1:len(test.columns)-1]
y_test = test.iloc[:, len(test.columns)-1]
X_test = pd.get_dummies(X_test)

In [ ]:
lgb.fit(X_train,y_train)

LGBMClassifier(is_unbalance=True)

In [ ]:
from sklearn.metrics import accuracy_score
pred=lgb.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))

Accuracy: 0.6313489896231568


In [ ]:
pred

array([1, 0, 0, ..., 1, 0, 0])

##Catboost

https://www.projectpro.io/recipes/use-catboost-classifier-and-regressor-in-python

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###data import and preprocessing

#### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

#### preprocessing(normalize)

In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-543-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-543-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


####split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


###fit

In [ ]:
import catboost as ctb

In [ ]:
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)

In [ ]:
X_test = test.iloc[:, 1:len(test.columns)-1]
y_test = test.iloc[:, len(test.columns)-1]
X_test = pd.get_dummies(X_test)

In [ ]:
model_CBC = ctb.CatBoostClassifier()
model_CBC.fit(X_train, y_train)

Learning rate set to 0.043577
0:	learn: 0.6432430	total: 21.3ms	remaining: 21.3s
1:	learn: 0.5988546	total: 32.4ms	remaining: 16.2s
2:	learn: 0.5594608	total: 39.8ms	remaining: 13.2s
3:	learn: 0.5244074	total: 47ms	remaining: 11.7s
4:	learn: 0.4932167	total: 54.7ms	remaining: 10.9s
5:	learn: 0.4655012	total: 61.7ms	remaining: 10.2s
6:	learn: 0.4408697	total: 71.6ms	remaining: 10.2s
7:	learn: 0.4189481	total: 81.7ms	remaining: 10.1s
8:	learn: 0.3994897	total: 87.3ms	remaining: 9.62s
9:	learn: 0.3822194	total: 93ms	remaining: 9.21s
10:	learn: 0.3668439	total: 98.6ms	remaining: 8.86s
11:	learn: 0.3531411	total: 106ms	remaining: 8.77s
12:	learn: 0.3408187	total: 114ms	remaining: 8.64s
13:	learn: 0.3299574	total: 121ms	remaining: 8.53s
14:	learn: 0.3202790	total: 127ms	remaining: 8.34s
15:	learn: 0.3115694	total: 135ms	remaining: 8.27s
16:	learn: 0.3038461	total: 141ms	remaining: 8.18s
17:	learn: 0.2969361	total: 149ms	remaining: 8.11s
18:	learn: 0.2907616	total: 154ms	remaining: 7.94s
19:	

In [ ]:
expected_y  = y_test
predicted_y = model_CBC.predict(X_test)
def accuracy(y, y1):
  right = 0
  wrong = 0
  for i in range(len(y)):
    if y[i] == y1[i]:
      right += 1
    elif y[i] != y1[i]:
      wrong += 1
  acc = right/(right+wrong)
  return acc
accuracy(expected_y,predicted_y)

0.9339158929546696

In [ ]:
predicted_y

array([0, 0, 0, ..., 0, 0, 0])

#K-fold

##Random_Forest(by scikit learn)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)
forest = RandomForestClassifier()

###n = 3

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(forest, X_train, y_train, cv=3, scoring="accuracy")
print(accuracy)

[0.91767356 0.91920123 0.91991807]


###n = 5

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(forest, X_train, y_train, cv=5, scoring="accuracy")
print(accuracy)

[0.9162116  0.91449053 0.91807476 0.92097628 0.91824543]


###n = 10

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(forest, X_train, y_train, cv=10, scoring="accuracy")
print(accuracy)

[0.91774744 0.91877133 0.91365188 0.91706485 0.92150171 0.91911263
 0.92318197 0.91703653 0.91806077 0.92045067]


##XGBoost

In [ ]:
from xgboost import XGBClassifier
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)

###n = 3

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(xgboostModel, X_train, y_train, cv=3, scoring="accuracy")
print(accuracy)

[0.93661683 0.93691756 0.93671275]


###n = 5

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(xgboostModel, X_train, y_train, cv=5, scoring="accuracy")
print(accuracy)

[0.93686007 0.93684929 0.93684929 0.93684929 0.93684929]


###n = 10

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(xgboostModel, X_train, y_train, cv=10, scoring="accuracy")
print(accuracy)

[0.93686007 0.93686007 0.93686007 0.93686007 0.93686007 0.93686007
 0.9364971  0.93683851 0.93683851 0.93683851]


##lightGBM

In [ ]:
import lightgbm as lgb
lgb = lgb.LGBMClassifier(is_unbalance=True)
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)

###n = 3

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(lgb, X_train, y_train, cv=3, scoring="accuracy")
print(accuracy)

[0.63977063 0.64833589 0.64403482]


###n = 5

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(lgb, X_train, y_train, cv=5, scoring="accuracy")
print(accuracy)

[0.64317406 0.62331456 0.63952893 0.63082437 0.63833419]


###n = 10

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(lgb, X_train, y_train, cv=10, scoring="accuracy")
print(accuracy)

[0.62798635 0.62593857 0.63549488 0.63208191 0.62423208 0.61843003
 0.62342096 0.62581086 0.61625128 0.62068966]


##CatBoost

In [ ]:
import catboost as ctb
X_train = train.iloc[:, 1:len(train.columns)-1]
y_train = train.iloc[:, len(train.columns)-1]
X_train = pd.get_dummies(X_train)
model_CBC = ctb.CatBoostClassifier()

###n = 3

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(model_CBC, X_train, y_train, cv=3, scoring="accuracy")
print(accuracy)

Learning rate set to 0.036649
0:	learn: 0.6509103	total: 7.5ms	remaining: 7.5s
1:	learn: 0.6126204	total: 13.2ms	remaining: 6.59s
2:	learn: 0.5779909	total: 17.5ms	remaining: 5.8s
3:	learn: 0.5465774	total: 22.4ms	remaining: 5.59s
4:	learn: 0.5180982	total: 28.1ms	remaining: 5.6s
5:	learn: 0.4923129	total: 33.7ms	remaining: 5.58s
6:	learn: 0.4689212	total: 39.1ms	remaining: 5.54s
7:	learn: 0.4476827	total: 44.9ms	remaining: 5.57s
8:	learn: 0.4285021	total: 51.5ms	remaining: 5.67s
9:	learn: 0.4110322	total: 57.3ms	remaining: 5.67s
10:	learn: 0.3951536	total: 66.6ms	remaining: 5.99s
11:	learn: 0.3808588	total: 72.5ms	remaining: 5.97s
12:	learn: 0.3679453	total: 79ms	remaining: 6s
13:	learn: 0.3562388	total: 84.9ms	remaining: 5.98s
14:	learn: 0.3456029	total: 90.7ms	remaining: 5.96s
15:	learn: 0.3359782	total: 95.4ms	remaining: 5.87s
16:	learn: 0.3272187	total: 102ms	remaining: 5.87s
17:	learn: 0.3192482	total: 107ms	remaining: 5.86s
18:	learn: 0.3120099	total: 112ms	remaining: 5.79s
19:	

###n = 5

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(model_CBC, X_train, y_train, cv=5, scoring="accuracy")
print(accuracy)

串流輸出內容已截斷至最後 5000 行。
5:	learn: 0.4804036	total: 39.1ms	remaining: 6.47s
6:	learn: 0.4564427	total: 43.8ms	remaining: 6.21s
7:	learn: 0.4349206	total: 48.6ms	remaining: 6.03s
8:	learn: 0.4155726	total: 53.2ms	remaining: 5.86s
9:	learn: 0.3981673	total: 58.9ms	remaining: 5.83s
10:	learn: 0.3825442	total: 63.7ms	remaining: 5.73s
11:	learn: 0.3684705	total: 70.2ms	remaining: 5.78s
12:	learn: 0.3557909	total: 76.9ms	remaining: 5.84s
13:	learn: 0.3443330	total: 83.4ms	remaining: 5.87s
14:	learn: 0.3340470	total: 89.5ms	remaining: 5.88s
15:	learn: 0.3244884	total: 96.3ms	remaining: 5.92s
16:	learn: 0.3160599	total: 102ms	remaining: 5.88s
17:	learn: 0.3084347	total: 108ms	remaining: 5.91s
18:	learn: 0.3016677	total: 118ms	remaining: 6.11s
19:	learn: 0.2954844	total: 128ms	remaining: 6.26s
20:	learn: 0.2899756	total: 135ms	remaining: 6.28s
21:	learn: 0.2844570	total: 141ms	remaining: 6.28s
22:	learn: 0.2799463	total: 147ms	remaining: 6.23s
23:	learn: 0.2758560	total: 153ms	remaining: 6.23s
24:	

###n = 10

In [ ]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(model_CBC, X_train, y_train, cv=10, scoring="accuracy")
print(accuracy)

串流輸出內容已截斷至最後 5000 行。
6:	learn: 0.4481098	total: 47.2ms	remaining: 6.69s
7:	learn: 0.4263054	total: 54.3ms	remaining: 6.73s
8:	learn: 0.4068430	total: 61.4ms	remaining: 6.76s
9:	learn: 0.3894771	total: 72.6ms	remaining: 7.18s
10:	learn: 0.3739655	total: 86ms	remaining: 7.73s
11:	learn: 0.3600429	total: 95.6ms	remaining: 7.87s
12:	learn: 0.3473637	total: 103ms	remaining: 7.81s
13:	learn: 0.3362758	total: 108ms	remaining: 7.58s
14:	learn: 0.3262656	total: 115ms	remaining: 7.54s
15:	learn: 0.3173561	total: 120ms	remaining: 7.4s
16:	learn: 0.3093350	total: 126ms	remaining: 7.3s
17:	learn: 0.3021381	total: 134ms	remaining: 7.29s
18:	learn: 0.2956102	total: 140ms	remaining: 7.21s
19:	learn: 0.2897969	total: 147ms	remaining: 7.19s
20:	learn: 0.2845836	total: 153ms	remaining: 7.14s
21:	learn: 0.2798686	total: 159ms	remaining: 7.07s
22:	learn: 0.2756101	total: 166ms	remaining: 7.03s
23:	learn: 0.2717846	total: 172ms	remaining: 7.01s
24:	learn: 0.2682890	total: 179ms	remaining: 6.99s
25:	learn: 0

##function

In [1]:
def K_Fold(train, data_org, test, K, model_name):
  if model_name == Naive_Bayes_Classifier:
    test_data = test.iloc[:, :len(test.columns)-1]
  elif model_name == Random_Forest:
    test_data = test.iloc[:, 30:]
  num_val_samples = len(train) // K
  all_predict = {}
  X_train_data = train.iloc[:, :len(train.columns)-1]
  y_train_data = train.iloc[:, len(train.columns)-1] 
  for i in range(K):
    print('Processing fold #', i)
    X_val = X_train_data[i*num_val_samples : (i+1)*num_val_samples]
    y_val = y_train_data[i*num_val_samples : (i+1)*num_val_samples]

    X_train = pd.concat([X_train_data[:i*num_val_samples], X_train_data[(i+1)*num_val_samples:]], axis = 0)
    y_train = pd.concat([y_train_data[:i*num_val_samples], y_train_data[(i+1)*num_val_samples:]], axis = 0)

    train_data = pd.concat([X_train, y_train], axis = 1)
    model = model_name(train_data, data_org)
    model.fit()
    predict = model.predict(test_data)
    right = 0
    wrong = 0
    for ii in range(len(test_data)):
      if predict[ii] == test["is_claim"][ii]:
        right += 1
      else:
        wrong += 1
    print(f"accuracy of fold #{i} :", (right/(right+wrong)))
    all_predict[i] = predict
  data_predict = []
  for i in range(len(test_data)):
    p = []
    for ii in range(K):
      p.append(all_predict[ii][i])
    data_predict.append(max(p, key = p.count))
  print(f"return the predict by {K}_times classify")
  return data_predict

##Bayes

###data import and preprocessing

#### load data

In [ ]:
data = pd.read_csv("train.csv")
data

,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58587,ID58588,0.355089,0.13,0.644231,C8,8794,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
58588,ID58589,1.199642,0.02,0.519231,C14,7788,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58589,ID58590,1.162273,0.05,0.451923,C5,34738,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
58590,ID58591,1.236307,0.14,0.557692,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0


In [ ]:
data["is_claim"].value_counts()

0    54844
1     3748
Name: is_claim, dtype: int64

#### preprocessing(normalize)

In [ ]:
data.iloc[:, 1].describe()
data.iloc[:, 1] = data.iloc[:, 1]/max(data.iloc[:, 1])
for i in range(len(data.iloc[:, 1])):
  if data.iloc[:, 1][i] >= 0.5:
    data.iloc[:, 1][i] = 1
  elif data.iloc[:, 1][i] < 0.5:
    data.iloc[:, 1][i] = 0    

<ipython-input-571-33ddf0c5afeb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 0
<ipython-input-571-33ddf0c5afeb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[:, 1][i] = 1


In [ ]:
data = data.replace("No", 0)
data = data.replace("Yes", 1)

####split data into train/test/validation part (2:1:1)

In [ ]:
index = np.arange(len(data))
np.random.shuffle(index)
data_org = data.iloc[:, 1:]
train = data.iloc[index[:int(len(data)/2)], 1:].reset_index(drop=True)
test = data.iloc[index[int(len(data)/2):int(len(data)*3/4)], 1:].reset_index(drop=True)
val = data.iloc[index[int(len(data)*3/4):], 1:].reset_index(drop=True)
print("train_len:", len(train), "\n", "test_len:", len(test), "\n", "tval_len:", len(val))

train_len: 29296 
 test_len: 14648 
 tval_len: 14648


###K fold

In [ ]:
predict = K_Fold(train, data_org, test, 3, Naive_Bayes_Classifier)